In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
])

In [ ]:
output_path = "/content/drive/MyDrive/Datasets/Ham10000/ham10000_spilt"
train_dataset = ImageFolder(os.path.join(output_path, "train"), transform= transform)
test_dataset = ImageFolder(os.path.join(output_path, "val"), transform= transform)
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Define a Linear Model

In [ ]:
num_inputs = 3 * 299 * 299
num_outputs = 7

class linearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(linearNet, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)
    def forward(self, x):
        y = self.linear(x.view(x.shape[0], -1))
        return y
linearnet = linearNet(num_inputs, num_outputs)
linearnet.to(device)
print(linearnet)

linearNet(
  (linear): Linear(in_features=268203, out_features=7, bias=True)
)


In [ ]:
# Define loss function and optimizer
import torch.nn.functional as F
import torch.optim as optim
learning_rate=0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(linearnet.parameters(),lr=learning_rate)

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for data in data_iter:
        X, y=data
        X, y=X.to(device),y.to(device)
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [ ]:
# Define trianing
epochs = 200
def train_model(optimizer):
    for epoch in range(epochs):
        train_loss_sum, train_acc_sum, n = 0.0, 0.0, 0
        for data in train_dataloader:
            inputs, y=data
            inputs, y=inputs.to(device),y.to(device)
            #print(inputs)
            y_hat = linearnet(inputs)
            cost = loss(y_hat, y).sum()

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            train_loss_sum += cost.item()
            train_acc_sum += (torch.argmax(y_hat, dim=1) == y).float().sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_dataloader, linearnet)
        print('epoch %d, loss %.4f, train acc %.4f, test acc %.4f' % (epoch + 1, train_loss_sum / n, train_acc_sum / n, test_acc))

In [ ]:
train_model(optimizer)

epoch 1, loss 0.5894, train acc 0.5107, test acc 0.1431
epoch 2, loss 0.4676, train acc 0.5390, test acc 0.1177
epoch 3, loss 0.4383, train acc 0.5434, test acc 0.2020
epoch 4, loss 0.3927, train acc 0.5537, test acc 0.6698
epoch 5, loss 0.3720, train acc 0.5619, test acc 0.6723
epoch 6, loss 0.3593, train acc 0.5600, test acc 0.1232
epoch 7, loss 0.3910, train acc 0.5596, test acc 0.4853
epoch 8, loss 0.3730, train acc 0.5644, test acc 0.5426
epoch 9, loss 0.3624, train acc 0.5688, test acc 0.5761
epoch 10, loss 0.3510, train acc 0.5638, test acc 0.3042
epoch 11, loss 0.3506, train acc 0.5670, test acc 0.6683
epoch 12, loss 0.3121, train acc 0.5805, test acc 0.5157
epoch 13, loss 0.3197, train acc 0.5788, test acc 0.5347
epoch 14, loss 0.3698, train acc 0.5657, test acc 0.6698
epoch 15, loss 0.3409, train acc 0.5742, test acc 0.6693
epoch 16, loss 0.3402, train acc 0.5779, test acc 0.6678
epoch 17, loss 0.3431, train acc 0.5745, test acc 0.6065
epoch 18, loss 0.3226, train acc 0.5767,